In [58]:
import pandas as pd
import json
import itertools
import matplotlib.pyplot as plt
import scipy.stats
%matplotlib inline

# Manipulando JSON

In [35]:
arquivo = open("categoriacd-export.json")
linhas = arquivo.readlines()
texto = "".join(linhas)
dados_fire = json.loads(texto)

In [36]:
for k,v in dados_fire["logs"].items():
    # print(k,v) # habilite para imprimir as chaves de mais alto nível
    for chave, valor in v.items():
        # print(chave, valor) # habilite para imprimir as chaves inferiores
        pass # desabilite quando reabilitar a linha de cima



In [37]:
lista_dicionarios = []

for k,v in dados_fire["logs"].items():
    lista_dicionarios.append(v)

        


In [38]:
dataframe = pd.DataFrame.from_records(lista_dicionarios)

In [39]:
dataframe

,tempo,tipo,user_id,versao
0,1479351735583,inicio,3137998881,A
1,1479345303606,final,3723105756,B
2,1478481355889,final,455822798,A
3,1478285321803,inicio,818354266,A
4,1479369743204,final,3936828694,A
5,1479345591168,inicio,978031372,B
6,1479347115072,final,3796913893,A
7,1479345504799,inicio,2419417467,A
8,1479378854855,inicio,4261560382,B
9,1478357257365,inicio,3729058087,A


In [40]:
agrupado = dataframe.groupby(by=["user_id", "versao"])

In [41]:
users = set(dataframe.user_id) # We convert it to a set so that each user appear only once

In [42]:
users = list(users) 

In [43]:
resultados = []

for u in users:
    for versao in ["A", "B"]:
        if (u, versao) in agrupado.groups.keys(): # Check if a specific pair user/type of test happened
            grupo1 =  agrupado.get_group((u, versao))
            grupo1 = grupo1.sort_values(by="tempo") # Para evitar que inícios sem fim gerem tempos negativos
            
            if len(grupo1[grupo1.tipo=="final"])>= 1 and len(grupo1[grupo1.tipo=="inicio"]) >= 1:
                delta_t = grupo1[grupo1.tipo=="final"].iloc[-1].tempo - grupo1[grupo1.tipo=="inicio"].iloc[0].tempo
                resultados.append({"user_id":u, "delta_t":delta_t, "versao":versao})

In [45]:
dataframe_resumo = pd.DataFrame.from_records(resultados)

In [46]:
dataframe_resumo.delta_t/=1000 # Convertendo milissegundos em segundos

In [63]:
dataframe_resumo

,delta_t,user_id,versao
0,20.348,3857193483,B
1,20.368,1686436492,B
2,127.473,2766741010,A
3,37.600,3936828694,A
4,28.412,542355478,B
5,50.978,2244328473,B
6,26.169,2991394204,A
7,973.810,3774183163,A
8,138.359,3774183163,B
9,16.863,1431168288,A


# Análise estatística

In [62]:
tempos_A = list(dataframe_resumo[dataframe_resumo.versao=="A"].delta_t)
tempos_B = list(dataframe_resumo[dataframe_resumo.versao=="B"].delta_t)

In [49]:
#filtrando resultados aberrantes
for i in tempos_A:
    if i < 0:
        tempos_A.remove(i)
    if i > 300:
        tempos_A.remove(i)
        
for i in tempos_B:
    if i < 0:
        tempos_B.remove(i)
    if i > 300:
        tempos_B.remove(i)
        

In [50]:
#calculando media, desvio padrao e variancia de A
somaA = 0
for i in tempos_A:
    somaA += i
mediaA = somaA / len(tempos_A)

somaA = 0
for i in tempos_A:
    somaA += (i - mediaA) ** 2
desvioA = (somaA / len(tempos_A)) ** (1 / 2)

varA = desvioA ** 2

print("A media é :{0}".format(mediaA))
print("O desvio é :{0}".format(desvioA))
print("A variância é :{0}".format(varA))

A media é :46.27719047619047
O desvio é :61.56936077351265
A variância é :3790.7861860589583


In [51]:
#calculando media, desvio padrao e variancia de B
somaB = 0
for i in tempos_B:
    somaB += i
mediaB = somaB / len(tempos_B)

somaB = 0
for i in tempos_B:
    somaB += (i - mediaB) ** 2
desvioB = (somaB / len(tempos_B)) ** (1 / 2)

varB = desvioB ** 2

print("A media é :{0}".format(mediaB))
print("O desvio é :{0}".format(desvioB))
print("A variância é :{0}".format(varB))

A media é :42.50672222222222
O desvio é :31.939199003393856
A variância é :1020.1124329783951


In [52]:
tempos_AB = []
for i in tempos_A:
    tempos_AB.append(i)
for i in tempos_B:
    tempos_AB.append(i)

#calculando media, desvio padrao e variancia de amostra total
somaAB = 0
for i in tempos_AB:
    somaAB += i
mediaAB = somaAB / len(tempos_AB)

somaAB = 0
for i in tempos_AB:
    somaAB += (i - mediaAB) ** 2
desvioAB = (somaAB / len(tempos_AB)) ** (1 / 2)

varAB = desvioAB ** 2

print("A media é :{0}".format(mediaAB))
print("O desvio é :{0}".format(desvioAB))
print("A variância é :{0}".format(varAB))

A media é :44.536974358974334
O desvio é :50.15522666809061
A variância é :2515.546762127548


In [ ]:
#definindo hipóteses
#H0 ; u <= 44.537
#Ha ; u > 44.537 
#95% de confiança

In [60]:
x = mediaB
u = mediaAB
s = desvioB
n = len(tempos_B)
#calculando graus de liberdade
v = n - 1
print(v)
#testando para ver se versao B rejeita a hipotese
t = (x - u) / (s / (n ** (1 / 2)))
print(t)
#calculando o valor-p
valorp = scipy.stats.t.cdf(t,v)
print(valorp)
#da tabela t student temos que para 95% de confiança com v graus de liberdade tcritico = 1.740. como estamos testando para 
#valores maiores que u0, tcritico esta na cauda da direita. Portanto não há evidências para refutar a H0

17
-0.269688363806
0.395323146076


In [61]:
x = mediaA
u = mediaAB
s = desvioA
n = len(tempos_A)
#calculando graus de liberdade
v = n - 1
print(v)
#testando para ver se versao A rejeita a hipotese
t = (x - u) / (s / (n ** (1 / 2)))
print(t)
#calculando o valor-p
valorp = scipy.stats.t.cdf(t,v)
print(valorp)
#da tabela t student temos que para 95% de confiança com v graus de liberdade tcritico = 1.740. como estamos testando para 
#valores maiores que u0, tcritico esta na cauda da direita. Portanto não há evidências para refutar a H0

20
0.129523386024
0.550881349379
